Créé le 12 mars 2021

**Projet Tableau de Bord** 

**Groupe n°3 - Arnaques en ligne**

**Nettoyage des données**


@authors:
- MANSON Marianne



#### Import libraries

In [ ]:
########## Module import ##########

# Fichiers
import json

# Format
import datetime

####  Récupération des données

In [ ]:
with open('raw_data.json') as json_data:
    data_dict = json.load(json_data)

#### Description des données  importées
 - data_dict est un dictionnaire de dictionnaire
 - Les clés sont des string d'entiers allant de 0 à 5999
 - Les valeurs sont des dictionnaires resprésentant les données d'un article
 - Ces données sont 'url', 'abstract', 'author', 'keyword', 'title' et 'date'
 - Cas de 'author' : c'est un dictionnaire qui associe un nom d'auteur à sa liste d'affiliations

#### Problemes à traiter 
- Abstract : symboles '\xa0' '\xa038'
- Abstract : cas de 'Abstract' collé au début du texte
- Auteur : symbole '\xa0' entre prénom et nom
- Auteur : université collé au nom de la ville et du pays (ex: 'University of HawaiiKiheiUSA')
- Auteur : cas d'un auteur vide dans le dictionnaire ('': [])
- Auteur : cas du numéro de l'affiliation ajouté
- Auteur : cas d'un "Email autho" collé à la fin de l'auteur
- keywords : '\xa0' à la fin des mots 

#### Partie Tests 

In [ ]:
nb_author_null = 0
nb_abstract_null = 0

print(data_dict['1'])

for key in data_dict.keys():
    article_dict = data_dict[key]
    abstract = article_dict['abstract']
    if abstract == "no_data" or abstract == 'ERREUR_REQUEST' or abstract == 'ERROR':
        nb_abstract_null+=1
    author_dict = article_dict['author']
    if not author_dict:
        nb_author_null+=1
    date_string = article_dict['date']
    try:
        date_object = datetime.datetime.strptime(date_string, "%Y-%m-%d")
    except ValueError:
        print(date_string)
        

print(nb_author_null)
print(nb_abstract_null)
print(len(data_dict))

{'url': 'https://link.springer.com/chapter/10.1007/978-3-319-58715-8_11', 'abstract': 'Understand the importance and scope of security of information systems for EC.', 'author': {'Efraim\xa0Turban': ['University of HawaiiKiheiUSA'], 'Jon\xa0Outland': ['Herzing UniversityRapid CityUSA'], 'David\xa0King': ['JDA SoftwareScottsdaleUSA'], 'Jae\xa0Kyu\xa0Lee': ['School of BusinessYonsei UniversitySeodaemun-guKorea'], 'Ting-Peng\xa0Liang': ['National Sun Yat-sen UniversityKaohsiungTaiwan'], 'Deborrah\xa0C.\xa0Turban': ['Turban Company Inc.KiheiUSA']}, 'keyword': ['Computer Fraud And Abuse Act (CFAA)\xa0', 'Ransomware\xa0', 'Botnet\xa0', 'Non-technical Methods\xa0', 'Malware\xa0'], 'title': 'E-Commerce Security and Fraud Issues and Protections', 'date': '2017-10-13'}
2008
2015
2016
2004
2011
2015
2015
2009
2016
2016
2013
2015
2014
2016
2016
2004
2016
2015
2014
2005
2006
2015
2013
2004
2014
2006-04
2014
2014
2016
2015
2016
2009
2016
2007
2007
2001
2016
2008
2014
2004-03
2004-12
2014
2005
2015
2

## Nettoyage des données 

#### Suppression des articles sans abstract

In [ ]:
# Suppression des articles sans abstract

new_data_dict = data_dict.copy()

print(len(new_data_dict))

for i in range(0,6000):
    if not (i == 190 or i == 1917): #Ces clés n'existent pas
        key = str(i)
        article_dict = new_data_dict[key]
        abstract = article_dict['abstract']
        if abstract == "no_data" or abstract == 'ERREUR_REQUEST' or abstract == 'ERROR':
            new_data_dict.pop(key)

print(len(new_data_dict))

5998
5629


#### nettoyage des dates

In [ ]:
# Changement des dates : mise au format datetime

for key in new_data_dict.keys():
    article_dict = new_data_dict[key]
    date_string = article_dict['date']
    if len(date_string) == 4:
        date_string += "-01-01"
    if len(date_string) == 7:
        date_string += "-01"
    try:
        date_object = datetime.datetime.strptime(date_string, "%Y-%m-%d")
        new_data_dict[key]['date'] = date_string
    except ValueError:
        print(date_string)
        

#### Nettoyage des auteurs

In [ ]:

nb_author_pb = 0

for key in new_data_dict.keys():
    article_dict = new_data_dict[key]
    author_dict = new_data_dict[key]['author'] # Attention, certains sont dans des listes, d'autres des dictionnaires
    if type(author_dict) == dict:
        for author in list(author_dict):
            if 'Email autho' in author:
                new_author = author.replace("Email autho", "")
                new_author = "".join([x if not x.isdigit() else "" for x in new_author])
                author_dict[new_author] = author_dict.pop(author)
                nb_author_pb+=1
            else:
                new_author = "".join([x if not x.isdigit() else "" for x in author]) 
                author_dict[new_author] = author_dict.pop(author)
                
    if type(author_dict) == list:
        for author in author_dict:
            author = "".join([x if not x.isdigit() else "" for x in author])
            if 'Email autho' in author:
                author.replace("Email autho", "")
                nb_author_pb+=1
                
        
print(nb_author_pb)
print(new_data_dict['0'])

0
{'url': 'https://link.springer.com/chapter/10.1007/978-3-319-15150-2_9', 'abstract': 'The term “419” is coined from Section\xa0419 of the Nigerian criminal code (part of Chapter\xa038: obtaining property by false pretenses; cheating) dealing with fraud.', 'author': {'Ashraf\xa0Darwish': ['Department of Computer Science, Faculty of ScienceHelwan UniversityCairoEgypt'], 'Mohammad\xa0Ayoub\xa0Khan': ['Department of Computer Science and Engineering, School of Engineering and TechnologySharda UniversityGreater NoidaIndia'], '': [], 'Mohamed\xa0Chawki': ['International Association of Cybercrime Prevention (AILCC)ParisFrance']}, 'keyword': ['Human Trafficking\xa0', 'Money Laundering\xa0', 'Criminal Code\xa0', 'Financial Crime\xa0', 'Unlawful Activity\xa0'], 'title': '419 Scam: An Evaluation of Cybercrime and Criminal Code in Nigeria', 'date': '2015-02-27'}


#### nettoyage de l'abstract


In [ ]:

nb_pb = 0

for key in new_data_dict.keys():
    article_dict = new_data_dict[key]
    abstract = article_dict['abstract']
    if abstract.startswith('Abstract'):
        nb_pb += 1
        new_data_dict[key]['abstract'] = abstract.replace('Abstract','',1)
        

print(nb_pb)

0


## Sauvegrade des données nettoyées

In [ ]:
with open('clean_articles.json', 'w') as f:
    json.dump(new_data_dict, f)